In [4]:
from bbq.analyse import *
import pandas as pd
from datetime import datetime, timedelta
from talib.abstract import *
import talib
import mplfinance as mpf
from bbq.data.stockdb import StockDB
from bbq.data.funddb import FundDB
from pyecharts.charts import *
from pyecharts import options as opts
from pyecharts.commons.utils import JsCode
import json
import scipy.signal as signal
from sklearn.linear_model import LinearRegression
import numpy as np
from bbq.analyse.tools import linear_fitting, extrema

db_stock = StockDB()
db_fund = FundDB()
db_stock.init()
db_fund.init()

data = await db_stock.load_stock_daily(filter={'code': 'sz002132'}, limit=60, sort=[('trade_date', -1)])
data = data[::-1]

name = await db_stock.load_stock_info(filter={'code': 'sz002132'}, projection=['name'])
data['name'] = name.iloc[0]['name']


data_fund = await db_fund.load_fund_daily(filter={'code': '159949'}, limit=60, sort=[('trade_date', -1)])
data_fund = data_fund[::-1]

name_fund = await db_fund.load_fund_info(filter={'code': '159949'}, projection=['short_name'])
data_fund['name'] = name_fund.iloc[0]['short_name']


2020-12-29 17:42:43,529 - StockDB - DEBUG - 加载股票日线, kwargs={'filter': {'code': 'sz002132'}, 'limit': 60, 'sort': [('trade_date', -1)]}
2020-12-29 17:42:43,593 - StockDB - DEBUG - 加载日线数据成功 size=60
2020-12-29 17:42:43,595 - StockDB - DEBUG - 加载股票信息, kwargs={'filter': {'code': 'sz002132'}, 'projection': ['name']} ...
2020-12-29 17:42:43,645 - StockDB - DEBUG - 加载股票信息成功 size=1
2020-12-29 17:42:43,649 - FundDB - DEBUG - 加载场内基金日线数据, kwargs={'filter': {'code': '159949'}, 'limit': 60, 'sort': [('trade_date', -1)]}
2020-12-29 17:42:43,917 - FundDB - DEBUG - 加载场内基金日线数据,成功, size=60
2020-12-29 17:42:43,918 - FundDB - DEBUG - 加载基金列表, kwargs={'filter': {'code': '159949'}, 'projection': ['short_name']}
2020-12-29 17:42:43,973 - FundDB - DEBUG - 加载基金列表成功, size=1


In [7]:

k_data = list(zip(data['open'], data['close'], data['low'], data['high']))
trade_date = [d.strftime('%Y-%m-%d') for d in data['trade_date']]

ma5 = MA(data['close'], timeperiod=5)
ma5 = [round(v, 3) for v in ma5]

wma5 = WMA(data['close'], timeperiod=5)
wma5 = [round(v, 3) for v in wma5]

line = Line()
line.add_xaxis(trade_date)
line.add_yaxis('ma5', ma5, label_opts=opts.LabelOpts(is_show=False), is_smooth=True, symbol='none')
line.add_yaxis('wma5', wma5, label_opts=opts.LabelOpts(is_show=False), is_smooth=True, symbol='none')
line.add_yaxis('close', data['close'], label_opts=opts.LabelOpts(is_show=False), is_smooth=True, symbol='none')

line.set_global_opts(xaxis_opts=opts.AxisOpts(is_scale=True),
                     yaxis_opts=opts.AxisOpts(
                         is_scale=True,
                         splitarea_opts=opts.SplitAreaOpts(
                             is_show=True, areastyle_opts=opts.AreaStyleOpts(opacity=1)
                         ),
                     ),
                      datazoom_opts=[opts.DataZoomOpts(pos_bottom="-2%", filter_mode='none')],
                      title_opts=opts.TitleOpts(title="Kline-MarkLine"))

x_index, _, y_extrema = extrema(data, cmp_func=np.greater)

scatter = Scatter()
scatter.add_xaxis(xaxis_data=x_index)
scatter.add_yaxis(
        series_name="极大值",
        y_axis=y_extrema,
        label_opts=opts.LabelOpts(is_show=False),
    )
line.overlap(scatter)

x_index, _, y_extrema = extrema(data, cmp_func=np.less)
scatter = Scatter()
scatter.add_xaxis(xaxis_data=x_index)
scatter.add_yaxis(
        series_name="极小值",
        y_axis=y_extrema,
        label_opts=opts.LabelOpts(is_show=False),
    )
line.overlap(scatter)

a, b, score, x_index, y_index = linear_fitting(data)

line2 = Line()
line2.add_xaxis(list(x_index))
line2.add_yaxis('线性拟合', y_index, label_opts=opts.LabelOpts(is_show=False), is_smooth=True)

line.overlap(line2)

line.render_notebook()


In [57]:
x = np.array(data['close'])
xmax = signal.argrelextrema(x, np.greater)
ymax = x[xmax[0]]
xmin = signal.argrelextrema(x, np.less)
ymin = x[xmin[0]]

#xmax = np.array(trade_date)[xmax[0]]
#xmin = np.array(trade_date)[xmin[0]]


#转换成numpy的ndarray数据格式，n行1列,LinearRegression需要列格式数据，如下：
X_train = np.array(xmax[0]).reshape((len(xmax[0]), 1))
Y_train = np.array(ymax).reshape((len(ymax), 1))
#新建一个线性回归模型，并把数据放进去对模型进行训练
lineModel = LinearRegression()
lineModel.fit(X_train, Y_train)

#用训练后的模型，进行预测
Y_predict = lineModel.predict(X_train)

#coef_是系数，intercept_是截距，拟合的直线是y=ax+b
a = lineModel.coef_[0][0]
b = lineModel.intercept_[0]
print("y=%.4f*x+%.4f" % (a,b))

#对回归模型进行评分，这里简单使用训练集进行评分，实际很多时候用其他的测试集进行评分
print("得分", lineModel.score(X_train, Y_train))

Y_predict

tt = [x for v in Y_predict for x in v]
tt


y=-0.0219*x+4.4635
得分 0.5510304368489158


[4.332257972408403,
 4.244790207688377,
 4.135455501788344,
 4.091721619428331,
 4.0261207958883105,
 3.9823869135282974,
 3.9386530311682844,
 3.8730522076282643,
 3.7855844429082377,
 3.719983619368218,
 3.654382795828198,
 3.588781972288178,
 3.501314207568152,
 3.4357133840281318,
 3.3919795016681187,
 3.2826447957680855,
 3.195177031048059]

In [41]:
ymin

array([4.24, 3.89, 3.89, 3.62, 3.63, 3.57, 3.47, 3.52, 3.76, 3.54, 3.5 ,
       3.39, 3.25, 3.23, 3.28, 3.29])

In [43]:
list(xmin)

['2020-10-19',
 '2020-10-23',
 '2020-10-28',
 '2020-10-30',
 '2020-11-04',
 '2020-11-06',
 '2020-11-11',
 '2020-11-13',
 '2020-11-20',
 '2020-11-25',
 '2020-11-30',
 '2020-12-03',
 '2020-12-09',
 '2020-12-11',
 '2020-12-16',
 '2020-12-22']